## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api

# Configure gmaps API key
gmaps.configure(api_key=google_api)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Chokurdakh,RU,70.6333,147.9167,-14.13,89,100,11.95,overcast clouds
1,1,Carnarvon,AU,-24.8667,113.6333,77.00,100,0,9.22,clear sky
2,2,Prince George,CA,53.9166,-122.7530,44.01,48,75,14.97,broken clouds
3,3,Georgetown,MY,5.4112,100.3354,79.00,94,20,4.61,few clouds
4,4,Hilo,US,19.7297,-155.0900,71.60,78,90,5.93,heavy intensity rain


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Carnarvon,AU,-24.8667,113.6333,77.00,100,0,9.22,clear sky
3,3,Georgetown,MY,5.4112,100.3354,79.00,94,20,4.61,few clouds
4,4,Hilo,US,19.7297,-155.0900,71.60,78,90,5.93,heavy intensity rain
5,5,Antofagasta,CL,-23.6500,-70.4000,73.00,73,20,17.27,few clouds
7,7,Faanui,PF,-16.4833,-151.7500,81.61,77,100,18.68,light rain
9,9,Port Elizabeth,ZA,-33.9180,25.5701,68.00,88,90,9.22,light rain
10,10,Butaritari,KI,3.0707,172.7902,80.56,79,0,8.16,clear sky
12,12,Arraial Do Cabo,BR,-22.9661,-42.0278,77.00,83,75,5.75,broken clouds
14,14,Aloleng,PH,16.1307,119.7824,76.78,75,24,7.07,few clouds
16,16,Avarua,CK,-21.2078,-159.7750,84.20,79,40,4.61,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                340
City                   340
Country                337
Lat                    340
Lng                    340
Max Temp               340
Humidity               340
Cloudiness             340
Wind Speed             340
Weather Description    340
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,
3,Georgetown,MY,79.00,few clouds,5.4112,100.3354,
4,Hilo,US,71.60,heavy intensity rain,19.7297,-155.0900,
5,Antofagasta,CL,73.00,few clouds,-23.6500,-70.4000,
7,Faanui,PF,81.61,light rain,-16.4833,-151.7500,
9,Port Elizabeth,ZA,68.00,light rain,-33.9180,25.5701,
10,Butaritari,KI,80.56,clear sky,3.0707,172.7902,
12,Arraial Do Cabo,BR,77.00,broken clouds,-22.9661,-42.0278,
14,Aloleng,PH,76.78,few clouds,16.1307,119.7824,
16,Avarua,CK,84.20,light rain,-21.2078,-159.7750,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,Hospitality Carnarvon
3,Georgetown,MY,79.00,few clouds,5.4112,100.3354,Cititel Penang
4,Hilo,US,71.60,heavy intensity rain,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Antofagasta,CL,73.00,few clouds,-23.6500,-70.4000,Holiday Inn Express Antofagasta
7,Faanui,PF,81.61,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
#8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Weather Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))